In [ ]:
from cling import cling, bash

**Listing 20.1**

Caption: Example for the usage function templates.

In [ ]:
%%cling
#include <cstdlib>
#include <string>
#include <iostream>
#include <complex>
//-----------------------------------------------------------------
// Definition of multiple functions
double add(double a, double b) { 
  return a + b;
}

std::string add(std::string a, std::string b) { 
  return a + b;
}

std::complex<int> add(std::complex<int> a, std::complex<int> b) {
  return a + b;
}

// Function template
template<typename T> 
T add_template(T a, T b){ 
  return a+b;
}

// Usage of the functions
std::cout << add(1.0, 1.5) << std::endl;
std::cout << add("hello ", "world") << std::endl;
std::cout << add(std::complex<int>(1,0),
    std::complex<int>(0,1)) << std::endl;
std::cout << "---------" << std::endl;
std::cout << add_template<float>(1.0, 1.5) << std::endl;
std::cout << add_template(
    std::complex<double>(1.1,0),
    std::complex<double>(.1,2.3)) << std::endl;


**Listing 20.3**

Caption: Printing the values of a \cpp{std::vector} using \cpp{std::for_each} and a lambda function.

In [ ]:
%%cling
#include <algorithm>
#include <iostream>
#include <vector>
//-----------------------------------------------------------------

// Vector to be printed
std::vector<int> values = {1, 2, 3, 4, 5}; 

// Define a named function 
void print(int value) {
  std::cout << value << std::endl;
}

// Print each element of the vector using a named function
std::for_each(values.begin(), values.end(), print); 

// Print each element of the vector using a unnamed function
std::for_each(values.begin(), values.end(), [](int value) {
  std::cout << value << std::endl;
}); 


**Listing 20.4**

Caption: Printing the values of a \cpp{std::vector} using \cpp{std::for_each} and a lambda function. Here, no pre-leading comma symbol is printed.

In [ ]:
%%writefile Listing_20_4.cpp
#include <algorithm>
#include <iostream>
#include <vector>
//-----------------------------------------------------------------

int main() {
  // Vector to be printed
  std::vector<int> v = {1,2,3,4,5};

  // Print the first element without a leading commas symbol
  std::for_each(v.begin(),v.end(),
    [init = true] (int x) mutable {
     if (init) { std::cout << x; init = false; }
     else { std::cout << ',' << x; }
  });
}


In [ ]:
%%bash
g++ -std=c++20 -I . -o Listing_20_4.exe Listing_20_4.cpp -lpthread
./Listing_20_4.exe

**Listing 20.5**

Caption: Example for move semantics.

In [ ]:
%%cling
#include <iostream>
#include <string>
#include <vector>
//-----------------------------------------------------------------
std::string name = "Parallel C++"; 
std::vector<std::string> values;

// Copy the object name to the end of the vector
values.push_back(name); 

std::cout << "The content of name after copying is " << name << std::endl; 

// Move the object name to the end of the vector
values.push_back(std::move(name)); 

std::cout << "The content of name after moving is " << name << std::endl; 


**Listing 20.6**

Caption: Example for the expression new and expression delete.

In [ ]:
%%cling
// Usage of the new expression to generate an array of integers
int *values = new int[5]; 

// Usage of the placement new for struct and classes
struct point { 
  double x,y,z;

  point (double x0=0, double y0=0, double z0=0) {
     x = x0; y = y0; z = z0;
  }
};

point *vec = new point(); 

// Releasing the allocate double array
delete[] values;  

// Releasing the allocated struct
delete vec; 


**Listing 20.7**

Caption: Usage of the unique pointer to allocate an array of double values.

In [ ]:
%%cling
#include <memory>
//-----------------------------------------------------------------
std::unique_ptr<double[]> a(new double[2]);

// Initialize the values
a[0] = 1; 
a[1] = 2;

// Generate a copy of the array a
// std::unique_ptr<double[]>b(a); 

// Generate a copy of the array a
std ::unique_ptr<double[]> b(std::move(a)); 


**Listing 20.8**

Caption: Usage of the shared pointer to allocate an array of double values.

In [ ]:
%%cling
#include <iostream>
#include <memory>
#include <array>
//-----------------------------------------------------------------
// Allocate a shared pointer to a double array 
std::shared_ptr<double[]> a (new double[2]);

// Initialize the value
a[0] = 1;
a[1] = 2;

// Create a copy of the reference to array a
std::shared_ptr<double[]> b(a); 

// Print the total references to the array
std::cout << a.use_count() << std::endl; 

// Release the smart pointer b
b.reset(); 

// Print the total references to the array
std::cout << a.use_count() << std::endl;  

// Create a copy of the reference to array a
//std :: unique_ptr < double [] > b ( a ) ; 


**Listing 20.9**

Caption: Example for finding all even values and compute their square root.

In [ ]:
%%cling
#include <cmath>
#include <iostream>
#include <vector>
//-----------------------------------------------------------------
std::vector<int> values = {0, 1, 2, 3, 4, 5, 6};

for (int i : values) {
  if (i % 2 == 0) {
    std::cout << std::sqrt(i) << " ";
  }
}


**Listing 20.10**

Caption: Example for finding all even values and compute their square root using the ranges library.

In [ ]:
%%writefile ranges.cpp
#include <cmath>
#include <iostream>
#include <ranges>
#include <vector>
//-----------------------------------------------------------------
int main() {
  std ::vector<int> values = {0, 1, 2, 3, 4, 5, 6};

  auto tr_values = values
    |  std::views::filter([](int value) { return value % 2 == 0; }) 
    |  std::views::transform([](int value) -> double { return std::sqrt(value);});
  for (int i : tr_values)
      std::cout << double(i) << ' ';
  std::cout << std::endl;
}


In [ ]:
%%bash
g++ -std=c++20 -I . -o ranges.exe ranges.cpp -lpthread
./ranges.exe

**Listing 21.1**

Caption: Configuration of default constants for the fractal sets and some utilities functions.

In [ ]:
%%writefile config.hpp
#ifndef CONFIG_HPP
#define CONFIG_HPP

#include <complex>
#include <string.h>
#include <cmath>
#include <iostream>
#include <sstream>

typedef std::complex<double> complex;
//-----------------------------------------------------------------

inline size_t get_size_t(const char *varname,size_t defval) {
    const char *strval = getenv(varname);
    size_t retval;
    if(strval == nullptr) {
        retval = defval;
    } else {
        std::stringstream ss(strval);
        ss >> retval;
    }
    std::cout << "Using " << varname << "=" << retval << std::endl;
    return retval;
}

inline std::string get_string(const char *varname,std::string defval) {
    const char *strval = getenv(varname);
    std::string retval;
    if(strval == nullptr)
        retval = defval;
    else
        retval = strval;
    std::cout << "Using " << varname << "=" << retval << std::endl;
    return retval;
}

inline std::string valid_string(std::string s) {
    if(s == "mandelbrot")
        ;
    else if(s == "julia")
        ;
    else {
        std::cout << "Only 'mandelbrot' or 'julia' are valid values for type.";
        exit(1);
    }
    return s;
}


// Definition of constants
const double pi = M_PI;
const size_t max_iterations = get_size_t("MAX_ITER",80);
const size_t size_x = get_size_t("SIZE_X",3840);
const size_t size_y = get_size_t("SIZE_Y",2160);
const int max_color = get_size_t("MAX_COLOR",256);
std::string type = valid_string(get_string("TYPE","mandelbrot"));
//-----------------------------------------------------------------
#endif


**Listing 21.2**

Caption: Utility to write Portable Bitmap File Format (PBM) files. This is used to store the images of the fractal sets.

In [ ]:
%%writefile pbm.hpp
#ifndef PBM_HPP
#define PBM_HPP

// Header for generating PBM image files
// https://en.wikipedia.org/wiki/Netpbm

#include <cassert>
#include <fstream>
#include <tuple>
#include <vector>

#include <config.hpp>
//-----------------------------------------------------------------
// Function to smoothen the coloring
std::tuple<size_t, size_t, size_t> get_rgb(int value) {
  double t = double(value) / double(max_iterations);
  int r = (int)(9 * (1 - t) * t * t * t * 255);
  int g = (int)(15 * (1 - t) * (1 - t) * t * t * 255);
  int b = (int)(8.5 * (1 - t) * (1 - t) * (1 - t) * t * 255);

  return std::make_tuple(r, g, b);
}
// convert an rgb tuple to an int
int make_color(int r, int g, int b) {
  assert(0 <= r && r < max_color);
  assert(0 <= g && g < max_color);
  assert(0 <= b && b < max_color);
  return (b * max_color + g) * max_color + r;
}

template<typename vector_type>
class PBM_ {
  // width and height
  int w, h;
  // a vector of vector if ints to store the pixels
  typename vector_type::allocator_type a;
  vector_type values;

  void _init() {
    // initialize vector to all zero
    for (int j = 0; j < h; j++) {
      std::vector<int> row;
      for (int i = 0; i < w; i++) {
        row.push_back(0);
      }
      values[j] = std::move(row);
    }
  }

 public:
  int width() { return w; }

  int height() { return h; }

  PBM_(){};

  PBM_(int w_, int h_) : w(w_), h(h_), a(), values(h_,a) { _init(); }
  PBM_(int w_, int h_, typename vector_type::allocator_type a_) : w(w_), h(h_), a(a_), values(h_,a) { _init(); }
  ~PBM_() {}

  // get or set a pixel at i, j
  int& operator()(int i, int j) { return values.at(j).at(i); }

  // get or set a row at j
  std::vector<int>& row(int j) { return values.at(j); }

  // save to a file
  void save(const std::string& fname) {
    std::ofstream f(fname);
    f << "P3\n";
    f << w << " " << h << "\n";
    f << max_color-1 << "\n";
    for (int j = 0; j < h; j++) {
      auto& row = values.at(j);
      for (int i = 0; i < w; i++) {
        int color = row.at(i);
        for (int c = 0; c < 3; c++) {
          f << (color % max_color) << " ";
          color /= max_color;
        }
        f << "\n";
      }
    }
    f.close();
  }
};

using PBM = PBM_<std::vector<std::vector<int>>>;
#endif


**Listing 21.3**

Caption: Utility to write Portable Bitmap File Format (PBM) files. This is used to store the images of the fractal sets.

In [ ]:
%%writefile kernel.hpp
#ifndef KERNEL_HPP
#define KERNEL_HPP

#include "pbm.hpp"
#include "config.hpp"
//-----------------------------------------------------------------

// Kernel to compute the Mandelbrot set
inline size_t mandelbrot(complex c) {
  std::complex<double> z(0, 0);
  for (size_t i = 0; i < max_iterations; i++) {
    z = z * z + c;
    if (abs(z) > 2.0) {
      return i;
    }
  }
  return 0;
}

inline double get_double(const char *varname,double defval) {
    const char *strval = getenv(varname);
    double retval;
    if(strval == nullptr) {
        retval = defval;
    } else {
        std::stringstream ss(strval);
        ss >> retval;
    }
    std::cout << "Using " << varname << "=" << retval << std::endl;
    return retval;
}

inline std::complex<double> get_const() {
  return {get_double("C_REAL",-.4),get_double("C_IMAG",.6)};
}

const std::complex<double> julia_const = get_const();

// Kernel to compute the Julia set
inline size_t julia(complex z) {
  for (size_t i = 0; i < max_iterations; i++) {
    z = z * z + julia_const;
    if (abs(z) > 2.0) {
      return i;
    }
  }
  return 0;
}

// Function to compute the Mandelbrot set for a pixel
inline size_t compute_pixel(complex c) {
  // std::complex<double> z(0, 0);
  for (size_t i = 0; i < max_iterations; i++) {
    if (type == "mandelbrot")
      return mandelbrot(c);
    else
      return julia(c);
  }

  return 0;
}

std::vector<int> compute_row(int item_index) {

  std::vector<int> result (size_y);

  complex c = complex(0, 4) * complex(item_index, 0) /
    complex(size_x, 0) - complex(0, 2);

  for (int i = 0; i < size_y; i++) {
    int value = compute_pixel(c + 4.0 * i / size_y - 2.0);
    std::tuple<size_t, size_t, size_t> color = get_rgb(value);
    result[i] = make_color(std::get<0>(color),
                           std::get<1>(color),
                           std::get<2>(color));
  }

  return std::move(result);
}
#endif


**Listing 21.4**

Caption: Utility functions for the Message Passing Interface.

In [ ]:
%%writefile util.hpp
#ifndef UTIL_HPP
#define UTIL_HPP

// Split work evenly as possible. If the work cannot be
// divided exactly equally, then then first few workers
// will have one more item than the remainder.
void split_work(std::size_t workers, std::size_t work_items,
                std::size_t worker_num, std::size_t &start,
                std::size_t &end) {
  std::size_t partition_size = work_items / workers;
  std::size_t excess = work_items % workers;
  start = partition_size *  worker_num    + (std::min)(worker_num  , excess);
  end   = partition_size * (worker_num+1) + (std::min)(worker_num+1, excess);
}
#endif
